Basic pedestrians crossing dataset generation + pose projection overlay
=======================================================================

Random generators are seeded, so unless code/settings are changed, the same results will be obtained.

In [ ]:
OUTPUT_DIR = '/outputs/scenarios/datasets/crossing'

In [ ]:
import os
import shutil

if os.path.exists(OUTPUT_DIR):
    shutil.rmtree(OUTPUT_DIR)

In [ ]:
from pedestrians_scenarios.skeleton import main

main([
    f"--outputs-dir={OUTPUT_DIR}",
    "--number-of-clips=8",
    "--clip-length-in-frames=600",
    "--batch-size=4",
])

In [ ]:
import pandas as pd
import ast

from pedestrians_scenarios.karma.utils.conversions import convert_flat_list_to_vector3d, convert_flat_list_to_transform
from pedestrians_scenarios.karma.pose.pose_dict import convert_flat_list_to_pose_dict, convert_flat_list_to_pose_2d_dict

dataset = pd.read_csv(
    os.path.join(OUTPUT_DIR, 'data.csv'),
    index_col=['id', 'camera.idx', 'frame.idx', 'pedestrian.idx'],
    converters={
        'camera.transform': lambda x: convert_flat_list_to_transform(ast.literal_eval(x)),
        'pedestrian.spawn_point': lambda x: convert_flat_list_to_transform(ast.literal_eval(x)),
        'frame.pedestrian.transform': lambda x: convert_flat_list_to_transform(ast.literal_eval(x)),
        'frame.pedestrian.velocity': lambda x: convert_flat_list_to_vector3d(ast.literal_eval(x)),
        'frame.pedestrian.pose.world': lambda x: convert_flat_list_to_pose_dict(ast.literal_eval(x)),
        'frame.pedestrian.pose.component': lambda x: convert_flat_list_to_pose_dict(ast.literal_eval(x)),
        'frame.pedestrian.pose.relative': lambda x: convert_flat_list_to_pose_dict(ast.literal_eval(x)),
        'frame.camera.pose': lambda x: convert_flat_list_to_pose_2d_dict(ast.literal_eval(x))
    }
)

In [ ]:
# get the data for the first pedestrian in the clips' 300th frame for now - this will get us one frame for each clip

frame_idx = 250
some_frames = dataset.loc[(slice(None), 0, frame_idx, 0), slice(None)]
some_frames

In [ ]:
# extract the correct frame images from the recordings

import pims

canvases = []
for (index_label, row_series) in some_frames.iterrows():
    video_path = os.path.join(OUTPUT_DIR, row_series['camera.recording'])
    video = pims.PyAVReaderTimed(video_path)
    frame_canvas = video[frame_idx]
    canvases.append(frame_canvas)

In [ ]:
# drawing method extracted from pedestrians-video-2-carla PointsRenderer

import numpy as np
from PIL import Image, ImageDraw

def draw_projection_points(canvas, points, skeleton, color_values=None, lines=False):
    """
    Draws the points on the copy of the canvas.
    """
    rounded_points = np.round(points).astype(int)

    end = canvas.shape[-1]
    has_alpha = end == 4
    img = Image.fromarray(canvas, 'RGBA' if has_alpha else 'RGB')
    draw = ImageDraw.Draw(img, 'RGBA' if has_alpha else 'RGB')

    if color_values is None:
        color_values = list(skeleton.get_colors().values())

    # if we know that skeleton has root point, we can draw it
    root_idx = skeleton.get_root_point() if skeleton is not None else None
    if root_idx is not None:
        draw.rectangle(
            [tuple(rounded_points[0] - 2), tuple(rounded_points[0] + 2)],
            fill=color_values[0][:end],
            outline=None
        )

    for idx, point in enumerate(rounded_points):
        if idx == root_idx:
            continue
        draw.ellipse(
            [tuple(point - 2), tuple(point + 2)],
            fill=color_values[idx][:end],
            outline=None
        )

    if lines and hasattr(skeleton, 'get_edges'):
        height, width = canvas.shape[:2]
        edges = skeleton.get_edges()
        for edge in edges:
            line_start = tuple(rounded_points[edge[0].value])
            line_end = tuple(rounded_points[edge[1].value])
            # skip line if either one of the points is not visible
            if not (line_start[0] > 0 and line_start[1] > 0 and line_end[0] > 0 and line_end[1] > 0
                    and line_start[0] < width and line_start[1] < height and line_end[0] < width and line_end[1] < height):
                continue
            draw.line(
                [line_start, line_end],
                fill=color_values[edge[0].value][:end],
                width=2
            )

    return np.array(img)

In [ ]:
# for each frame, overlay the pose on the image

from pedestrians_scenarios.karma.pose.skeleton import CARLA_SKELETON

class MockCamera(object):
    attributes = {
        'image_size_x': 800,
        'image_size_y': 600,
        'fov': 90.0,
        'lens_x_size': 0.08,
        'lens_y_size': 0.08,
    }

drawings = []
for (index_label, row_series), frame_canvas in zip(some_frames.iterrows(), canvases):
    projection = np.array([[v.x, v.y] for v in row_series['frame.camera.pose'].values()], dtype=np.float32)
    drawing = draw_projection_points(frame_canvas, projection, CARLA_SKELETON, None, lines=True)
    drawings.append(Image.fromarray(drawing))
    

In [ ]:
from IPython.display import display

for drawing in drawings:
    display(drawing)